In [ ]:
import os
from pathlib import Path
import subprocess
import json

import pandas as pd
from notion_client import Client
from transformers import pipeline

from config import (
    CANDIDATE_LABELS,
    DATABASE_ID,
    NOTION_ACCESS_TOKEN,
    GYAZO_ACCESS_TOKEN,
    CACHE_PATH,
    ARXIV_CATEGORIES,
)
from lib import get_web_content, label_text, post_to_notion

In [ ]:
URL = "https://arxiv.org/abs/2301.02654"

In [ ]:
notion = Client(auth=NOTION_ACCESS_TOKEN)

In [ ]:
p = get_web_content(url=URL, cache_path=CACHE_PATH, arxiv_categories=ARXIV_CATEGORIES)

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
import re
from pprint import pprint

In [ ]:
# html_content = re.sub(r"\(<a.*?</a>\)", " ", p.html_content)

# try:
#     html_content = html_content.rsplit('<section id="bib"')[0]
# except Exception:
#     pass

# html_content = '<section id="S1"' + html_content.split('<section id="S1"')[1]

# soup = BeautifulSoup(p.html_content, "html.parser")
    
    
# for error_span in soup.find_all("span", class_="ltx_ERROR"):
#     error_span.decompose()

# for section in soup.find_all("section"):
#     paragraphs = []
#     for para in section.find_all("div", class_="ltx_para", recursive=True):
#         paragraphs.append(para.text)
#         para.decompose()
#     new_para = soup.new_tag("p", attrs={"class": "ltx_para"})
#     new_para.string = "\n".join(paragraphs)
#     section.append(new_para)
    
# for sub_section in soup.find_all("section", class_="ltx_subsection"):
#     sub_section.decompose()

In [ ]:
import arxiv

In [ ]:
soup = BeautifulSoup(p.html_content, "html.parser")

# get only figures
figures = []
for fig in soup.find_all("figure"):
    if not len(fig.find_all("td")) > 0:
        figures.append(fig)

In [ ]:
res = next(arxiv.Search(id_list=[URL.split("/")[-1]]).results()).summary

In [ ]:
res

In [ ]:
html = f"""
<html>
    <body>
        <h2>Abstract</h2>
            <p>{res}</p>
        <h2>Figures</h2>
        {''.join(list(map(lambda x: str(x), figures)))}
    </body>
</html>
"""

In [ ]:
fugu_translator = pipeline('translation', model='staka/fugumt-en-ja')

In [ ]:
for para in soup.find_all("p", class_="ltx_para", recursive=True):
    para.string = fugu_translator(para.text)[0]['translation_text']

In [ ]:
with open("/home/workspace/notion-auto-archive/content/i.html", "w") as f:
    f.write(html)

In [ ]:
if p.tags is None:
    p.tags = label_text(
        text=p.cleansed_content,
        candidate_labels=CANDIDATE_LABELS,
        threshold=0.9,
    )

In [ ]:
p.tags

In [ ]:
# import pickle
# pickle.dump(p, open("p.pkl", "wb"))

# import pickle
# p = pickle.loads(open("p.pkl", "rb").read())

In [ ]:
post_to_notion(
    notion_access_token=NOTION_ACCESS_TOKEN,
    gyazo_access_token=GYAZO_ACCESS_TOKEN,
    database_id=DATABASE_ID,
    processed_content=p,
)